## Setup

In [1]:
import pandas as pd

In [2]:
baseball = pd.read_csv('data/baseball.csv')
baseball = baseball.drop(['Name', 'Age', 'Name-additional'], axis = 1)
baseball['Salary'] = baseball['Salary'].str.replace('$', '').astype(float)

baseball['C'] = baseball['Position'].apply(lambda x: 1 if 'C' in x else 0)
baseball['1B'] = baseball['Position'].apply(lambda x: 1 if '1B' in x else 0)
baseball['2B'] = baseball['Position'].apply(lambda x: 1 if '2B' in x else 0)
baseball['3B'] = baseball['Position'].apply(lambda x: 1 if '3B' in x else 0)
baseball['SS'] = baseball['Position'].apply(lambda x: 1 if 'SS' in x else 0)
baseball['OF'] = baseball['Position'].apply(lambda x: 1 if 'OF' in x else 0)

baseball['Num_Pos'] = baseball[['C', '1B', '2B', '3B', 'SS', 'OF']].sum(axis = 1)
baseball = baseball.drop(['Position'], axis = 1)

In [3]:
# packages used in each section below
from sklearn.feature_selection import RFE
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import numpy as np

import shap
from sklearn.decomposition import PCA

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [4]:
X = baseball.drop(['Salary'], axis = 1)
y = baseball['Salary']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .25, random_state = 621)

In [5]:
cat_columns = ['Tm', 'Lg', 'Acquired', 'Bat']
num_columns = [col for col in X.columns if col not in cat_columns + ['C', '1B', '2B', '3B', 'SS', 'OF']]

cat_transformer = Pipeline(
    steps = [
        ('onehot', OneHotEncoder(handle_unknown = 'ignore'))
    ]
)

num_transformer = Pipeline(
    steps = [
        ('scale', StandardScaler())
    ]
)

preprocessor = ColumnTransformer(
    transformers = [
        ('cont', num_transformer, num_columns),
        ('cat', cat_transformer, cat_columns)
    ], remainder = 'passthrough'
)

X_transform = preprocessor.fit_transform(X)

selected_features = np.concatenate([
    np.array(num_columns),
    np.array(preprocessor.transformers_[1][1]['onehot'].get_feature_names_out(cat_columns)),
    np.array(['C', '1B', '2B', '3B', 'SS', 'OF'])
])


## Random Forest

In [6]:
from sklearn.ensemble import RandomForestRegressor

In [7]:
rfe_rf = RFE(estimator = RandomForestRegressor(random_state = 621), step = 1, n_features_to_select = None)
rfe_rf.fit(X_transform, y)

RFE(estimator=RandomForestRegressor(random_state=621))

In [8]:
selected_features[rfe_rf.support_]

array(['Def-Inn', 'PO', 'A', 'E', 'DP', 'Fld%', 'Rdrs', 'Season', 'RAA',
       'WAA', 'RAR', 'WAR', 'PA', 'AB', 'R', 'H', 'HR', 'RBI', 'SB', 'CS',
       'BB', 'SO', 'BA', 'OBP', 'SLG', 'OPS', 'OPS+', 'TB', 'GDP', 'HBP',
       'SH', 'SF', 'IBB', 'Num_Pos', 'Tm_LAA', 'Tm_LAD', 'Tm_MULTIPLE',
       'Tm_NYM', 'Tm_NYY', 'Tm_SFG', 'Tm_STL', 'Acquired_Free Agency',
       'Acquired_Traded'], dtype=object)

In [123]:
red_num_columns = ['Def-Inn', 'PO', 'A', 'E', 'DP', 'Fld%', 'Rdrs', 'RAA', 'WAA', 'RAR',
               'WAR', 'PA', 'AB', 'R', 'H', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'BA', 'OBP',
               'SLG', 'OPS', 'OPS+', 'TB', 'GDP', 'HBP', 'SH', 'SF', 'IBB', 'Num_Pos', 'Season']
red_cat_columns = ['Tm', 'Acquired']
X_rf = X[red_num_columns + red_cat_columns]

In [124]:
X_rf_red_train, X_rf_red_test, y_rf_red_train, y_rf_red_test = train_test_split(X_rf, y, test_size = .25, random_state = 621)

In [148]:
red_preprocessor = ColumnTransformer(
    transformers = [
        ('cont', num_transformer, red_num_columns),
        ('cat', cat_transformer, red_cat_columns)
    ], remainder = 'passthrough'
)

rf_red_pipe = Pipeline(
    steps = [
        ('preprocessor', red_preprocessor),
        ('model', RandomForestRegressor(n_estimators = 200, max_depth = 12, min_samples_split = 3))
    ]
)

In [149]:
rf_red_pipe.fit(X_rf_red_train, y_rf_red_train)
rf_red_train_mse = mean_squared_error(y_rf_red_train, rf_red_pipe.predict(X_rf_red_train))
rf_red_test_mse = mean_squared_error(y_rf_red_test, rf_red_pipe.predict(X_rf_red_test))
print('Reduced Random Forest Metrics:')
print(f'Training RMSE: {np.sqrt(rf_red_train_mse)}')
print(f'Test RMSE: {np.sqrt(rf_red_test_mse)}')

Reduced Random Forest Metrics:
Training RMSE: 2665873.0847510966
Test RMSE: 5328864.974109751


In [116]:
rf_pipe = Pipeline(
    steps = [
        ('preprocessor', preprocessor),
        ('model', RandomForestRegressor(n_estimators = 200, min_samples_split = 14))
    ]
)

In [117]:
rf_pipe.fit(X_train, y_train)
rf_train_mse = mean_squared_error(y_train, rf_pipe.predict(X_train))
rf_test_mse = mean_squared_error(y_test, rf_pipe.predict(X_test))
print('Random Forest Metrics:')
print(f'Training RMSE: {np.sqrt(rf_train_mse)}')
print(f'Test RMSE: {np.sqrt(rf_test_mse)}')

Random Forest Metrics:
Training RMSE: 2995640.073838007
Test RMSE: 5339371.913062029


## XGBoost 

In [15]:
from xgboost import XGBRegressor

In [16]:
rfe_xg = RFE(estimator = XGBRegressor(random_state = 621), step = 1, n_features_to_select = None)
rfe_xg.fit(X_transform, y)

RFE(estimator=XGBRegressor(base_score=None, booster=None, callbacks=None,
                           colsample_bylevel=None, colsample_bynode=None,
                           colsample_bytree=None, early_stopping_rounds=None,
                           enable_categorical=False, eval_metric=None,
                           feature_types=None, gamma=None, gpu_id=None,
                           grow_policy=None, importance_type=None,
                           interaction_constraints=None, learning_rate=None,
                           max_bin=None, max_cat_threshold=None,
                           max_cat_to_onehot=None, max_delta_step=None,
                           max_depth=None, max_leaves=None,
                           min_child_weight=None, missing=nan,
                           monotone_constraints=None, n_estimators=100,
                           n_jobs=None, num_parallel_tree=None, predictor=None,
                           random_state=621, ...))

In [17]:
selected_features[rfe_xg.support_]

array(['Season', 'RBI', 'CS', 'BB', 'SO', 'BA', 'OBP', 'SLG', 'OPS+',
       'TB', 'GDP', 'HBP', 'SH', 'SF', 'IBB', 'Num_Pos', 'Tm_ARI',
       'Tm_BOS', 'Tm_CHC', 'Tm_CIN', 'Tm_DET', 'Tm_HOU', 'Tm_KCR',
       'Tm_LAA', 'Tm_LAD', 'Tm_MIA', 'Tm_MIN', 'Tm_MULTIPLE', 'Tm_NYM',
       'Tm_NYY', 'Tm_OAK', 'Tm_PHI', 'Tm_STL', 'Tm_TBR', 'Tm_TEX',
       'Acquired_Amateur Draft', 'Acquired_Amateur Free Agent',
       'Acquired_Free Agency', 'Acquired_Traded', 'Bat_L', 'Bat_R', '1B',
       '2B'], dtype=object)

In [18]:
red_num_columns = ['Season', 'RBI', 'CS', 'BB', 'SO', 'BA', 'OBP', 'SLG', 'OPS+', 
                   'TB', 'GDP', 'HBP', 'SH', 'SF', 'IBB', 'Num_Pos']
red_cat_columns = ['Tm', 'Acquired', 'Bat', 'C', '1B', '2B', '3B', 'SS', 'OF']
X_xg = X[red_num_columns + red_cat_columns]

In [19]:
X_xg_red_train, X_xg_red_test, y_xg_red_train, y_xg_red_test = train_test_split(X_xg, y, test_size = .25, random_state = 621)

In [71]:
red_preprocessor = ColumnTransformer(
    transformers = [
        ('cont', num_transformer, red_num_columns),
        ('cat', cat_transformer, red_cat_columns)
    ], remainder = 'passthrough'
)

xg_red_pipe = Pipeline(
    steps = [
        ('preprocessor', red_preprocessor),
        ('model', XGBRegressor(n_estimators = 200, learning_rate = .06, max_depth = 5))
    ]
)

In [72]:
xg_red_pipe.fit(X_xg_red_train, y_xg_red_train)
xg_red_train_mse = mean_squared_error(y_xg_red_train, xg_red_pipe.predict(X_xg_red_train))
xg_red_test_mse = mean_squared_error(y_xg_red_test, xg_red_pipe.predict(X_xg_red_test))
print('Reduced XGBoost Metrics:')
print(f'Training RMSE: {np.sqrt(xg_red_train_mse)}')
print(f'Test RMSE: {np.sqrt(xg_red_test_mse)}')

Reduced Random Forest Metrics:
Training RMSE: 3174834.536729983
Test RMSE: 5035866.184550909


In [155]:
xg_pipe = Pipeline(
    steps = [
        ('preprocessor', preprocessor),
        ('model', XGBRegressor(n_estimators = 100, learning_rate = .03, max_depth = 6))
    ]
)

In [156]:
xg_pipe.fit(X_train, y_train)
xg_train_mse = mean_squared_error(y_train, xg_pipe.predict(X_train))
xg_test_mse = mean_squared_error(y_test, xg_pipe.predict(X_test))
print('XGBoost Metrics:')
print(f'Training RMSE: {np.sqrt(xg_train_mse)}')
print(f'Test RMSE: {np.sqrt(xg_test_mse)}')

Random Forest Metrics:
Training RMSE: 3537922.5378611186
Test RMSE: 5184611.048067813


## Gradient Boosting

In [157]:
from sklearn.ensemble import GradientBoostingRegressor

In [158]:
rfe_gb = RFE(estimator = GradientBoostingRegressor(random_state = 621), step = 1, n_features_to_select = None)
rfe_gb.fit(X_transform, y)

RFE(estimator=GradientBoostingRegressor(random_state=621))

In [159]:
selected_features[rfe_gb.support_]

array(['Def-Inn', 'PO', 'A', 'E', 'DP', 'Fld%', 'Rdrs', 'Season', 'RAA',
       'WAA', 'RAR', 'WAR', 'PA', 'AB', 'R', 'H', 'HR', 'RBI', 'SB', 'CS',
       'BB', 'SO', 'BA', 'OBP', 'SLG', 'OPS', 'OPS+', 'TB', 'GDP', 'SH',
       'SF', 'IBB', 'Num_Pos', 'Tm_LAA', 'Tm_LAD', 'Tm_MIN',
       'Tm_MULTIPLE', 'Tm_NYY', 'Tm_OAK', 'Tm_STL',
       'Acquired_Free Agency', 'Acquired_Traded', '2B'], dtype=object)

In [160]:
red_num_columns = ['Def-Inn', 'PO', 'A', 'E', 'DP', 'Fld%', 'Rdrs', 'Season', 'RAA',
       'WAA', 'RAR', 'WAR', 'PA', 'AB', 'R', 'H', 'HR', 'RBI', 'SB', 'CS',
       'BB', 'SO', 'BA', 'OBP', 'SLG', 'OPS', 'OPS+', 'TB', 'GDP', 'SH',
       'SF', 'IBB', 'Num_Pos']
red_cat_columns = ['Tm', 'Acquired', 'C', '1B', '2B', '3B', 'SS', 'OF']
X_gb = X[red_num_columns + red_cat_columns]

In [161]:
X_gb_red_train, X_gb_red_test, y_gb_red_train, y_gb_red_test = train_test_split(X_gb, y, test_size = .25, random_state = 621)

In [196]:
red_preprocessor = ColumnTransformer(
    transformers = [
        ('cont', num_transformer, red_num_columns),
        ('cat', cat_transformer, red_cat_columns)
    ], remainder = 'passthrough'
)

gb_red_pipe = Pipeline(
    steps = [
        ('preprocessor', red_preprocessor),
        ('model', GradientBoostingRegressor(learning_rate = 0.075, n_estimators = 200, max_depth = 4))
    ]
)

In [197]:
gb_red_pipe.fit(X_gb_red_train, y_gb_red_train)
gb_red_train_mse = mean_squared_error(y_gb_red_train, gb_red_pipe.predict(X_gb_red_train))
gb_red_test_mse = mean_squared_error(y_gb_red_test, gb_red_pipe.predict(X_gb_red_test))
print('Reduced Gradient Boosting Metrics:')
print(f'Training RMSE: {np.sqrt(gb_red_train_mse)}')
print(f'Test RMSE: {np.sqrt(gb_red_test_mse)}')

Reduced Random Forest Metrics:
Training RMSE: 3426537.951924427
Test RMSE: 5028275.917521109


In [200]:
gb_pipe = Pipeline(
    steps = [
        ('preprocessor', preprocessor),
        ('model', GradientBoostingRegressor(learning_rate = 0.075, n_estimators = 200, max_depth = 4))
    ]
)

In [201]:
gb_pipe.fit(X_train, y_train)
gb_train_mse = mean_squared_error(y_train, gb_pipe.predict(X_train))
gb_test_mse = mean_squared_error(y_test, gb_pipe.predict(X_test))
print('Gradient Boosting Metrics:')
print(f'Training RMSE: {np.sqrt(gb_train_mse)}')
print(f'Test RMSE: {np.sqrt(gb_test_mse)}')

Random Forest Metrics:
Training RMSE: 3379850.3723368607
Test RMSE: 5028062.821060685


## ADA Boosting

In [202]:
from sklearn.ensemble import AdaBoostRegressor

In [203]:
rfe_ada = RFE(estimator = AdaBoostRegressor(random_state = 621), step = 1, n_features_to_select = None)
rfe_ada.fit(X_transform, y)

RFE(estimator=AdaBoostRegressor(random_state=621))

In [204]:
selected_features[rfe_ada.support_]

array(['Def-Inn', 'PO', 'A', 'E', 'DP', 'Fld%', 'Rdrs', 'Season', 'RAA',
       'WAA', 'RAR', 'WAR', 'PA', 'AB', 'R', 'H', 'HR', 'RBI', 'CS', 'BB',
       'SO', 'BA', 'OBP', 'SLG', 'OPS', 'OPS+', 'GDP', 'HBP', 'SH', 'SF',
       'IBB', 'Num_Pos', 'Tm_CHC', 'Tm_COL', 'Tm_DET', 'Tm_HOU', 'Tm_LAA',
       'Tm_NYM', 'Tm_NYY', 'Acquired_Free Agency', 'Bat_L', 'Bat_R', '2B'],
      dtype=object)

In [205]:
red_num_columns = ['Def-Inn', 'PO', 'A', 'E', 'DP', 'Fld%', 'Rdrs', 'Season', 'RAA', 'WAA',
       'RAR', 'WAR', 'PA', 'AB', 'R', 'H', 'HR', 'RBI', 'CS', 'BB', 'SO', 'BA',
       'OBP', 'SLG', 'OPS', 'OPS+', 'GDP', 'HBP', 'SH', 'SF', 'IBB',
       'Num_Pos']
red_cat_columns = ['Tm', 'Acquired', 'Bat', 'C', '1B', '2B', '3B', 'SS', 'OF']
X_ada = X[red_num_columns + red_cat_columns]

In [206]:
X_ada_red_train, X_ada_red_test, y_ada_red_train, y_ada_red_test = train_test_split(X_ada, y, test_size = .25, random_state = 621)

In [246]:
red_preprocessor = ColumnTransformer(
    transformers = [
        ('cont', num_transformer, red_num_columns),
        ('cat', cat_transformer, red_cat_columns)
    ], remainder = 'passthrough'
)

ada_red_pipe = Pipeline(
    steps = [
        ('preprocessor', red_preprocessor),
        ('model', AdaBoostRegressor(n_estimators = 200, learning_rate = 0.01))
    ]
)

In [247]:
ada_red_pipe.fit(X_ada_red_train, y_ada_red_train)
ada_red_train_mse = mean_squared_error(y_ada_red_train, ada_red_pipe.predict(X_ada_red_train))
ada_red_test_mse = mean_squared_error(y_ada_red_test, ada_red_pipe.predict(X_ada_red_test))
print('Reduced ADABoost Metrics:')
print(f'Training RMSE: {np.sqrt(ada_red_train_mse)}')
print(f'Test RMSE: {np.sqrt(ada_red_test_mse)}')

Reduced Random Forest Metrics:
Training RMSE: 5358808.636299628
Test RMSE: 5656509.686938622


In [252]:
ada_pipe = Pipeline(
    steps = [
        ('preprocessor', preprocessor),
        ('pca', PCA(n_components = 30)),
        ('model', AdaBoostRegressor(n_estimators = 200, learning_rate = 0.01))
    ]
)

In [253]:
ada_pipe.fit(X_train, y_train)
ada_train_mse = mean_squared_error(y_train, ada_pipe.predict(X_train))
ada_test_mse = mean_squared_error(y_test, ada_pipe.predict(X_test))
print('ADABoost Metrics:')
print(f'Training RMSE: {np.sqrt(ada_train_mse)}')
print(f'Test RMSE: {np.sqrt(ada_test_mse)}')

Random Forest Metrics:
Training RMSE: 5528645.110368226
Test RMSE: 5866138.083422676


## Support Vector Machine

In [254]:
from sklearn.svm import SVC

In [263]:
svm_pipe = Pipeline(
    steps = [
        ('preprocessor', preprocessor),
        ('model', SVC(C = 3))
    ]
)

In [264]:
svm_pipe.fit(X_train, y_train)
svm_train_mse = mean_squared_error(y_train, svm_pipe.predict(X_train))
svm_test_mse = mean_squared_error(y_test, svm_pipe.predict(X_test))
print('Support Vector Machine Metrics:')
print(f'Training RMSE: {np.sqrt(svm_train_mse)}')
print(f'Test RMSE: {np.sqrt(svm_test_mse)}')

Random Forest Metrics:
Training RMSE: 3305791.485725613
Test RMSE: 6511994.651652397


## KNN

In [265]:
from sklearn.neighbors import KNeighborsRegressor

In [290]:
knn_pipe = Pipeline(
    steps = [
        ('preprocessor', preprocessor),
        ('pca', PCA(n_components = 63)),
        ('model', KNeighborsRegressor(n_neighbors = 5, weights = 'uniform'))
    ]
)

In [291]:
knn_pipe.fit(X_train, y_train)
knn_train_mse = mean_squared_error(y_train, knn_pipe.predict(X_train))
knn_test_mse = mean_squared_error(y_test, knn_pipe.predict(X_test))
print('K-Nearest Neighbors Metrics:')
print(f'Training RMSE: {np.sqrt(knn_train_mse)}')
print(f'Test RMSE: {np.sqrt(knn_test_mse)}')

Random Forest Metrics:
Training RMSE: 4357478.9227963155
Test RMSE: 5627354.873436867


## DNN

In [366]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dropout, Dense
from bayes_opt import BayesianOptimization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import warnings
import joblib

warnings.filterwarnings('ignore')


In [423]:
X_scaled = preprocessor.fit_transform(X)

pca = PCA(n_components = 45, random_state = 621)
X_pca = pca.fit_transform(X_scaled)

In [424]:
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size = .2, random_state = 621)

In [425]:
def dnn_model_score(neurons, dropout_rate, learning_rate, epochs, batch_size, patience, num_layers, **layer_neurons):
    model = Sequential()
    model.add(Dense(int(neurons), activation='relu', input_shape = (X_train.shape[1],)))
    model.add(Dropout(dropout_rate))

    for i in range(1, int(num_layers) + 1):
        model.add(Dense(int(layer_neurons[f'layer_neurons_{i}']), activation='relu'))
        model.add(Dropout(dropout_rate))

    model.add(Dense(1, activation = 'linear'))

    optimizer = Adam(learning_rate = learning_rate)
    model.compile(loss = 'mean_squared_error', optimizer = optimizer)

    es = EarlyStopping(monitor = 'val_loss', patience = int(patience), restore_best_weights = True)

    model.fit(X_train, y_train, validation_split = .2, epochs = int(epochs), batch_size = int(batch_size), callbacks = es, verbose = 0)

    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    return -mse

pbounds = {'neurons': (32, 256),
           'dropout_rate': (0.0, 0.5),
           'learning_rate': (0.01, 1),
           'epochs' : (100, 500),
           'batch_size' : (32, 500),
           'patience' : (20, 50),
           'num_layers': (1, 5)}

for i in range(1, 6):
    pbounds[f'layer_neurons_{i}'] = (32, 256)

optimizer = BayesianOptimization(f = dnn_model_score, pbounds = pbounds, random_state = 42)

optimizer.maximize(init_points = 5, n_iter = 10)

best_params = optimizer.max['params']
print("Best Hyperparameters:", best_params)


|   iter    |  target   | batch_... | dropou... |  epochs   | layer_... | layer_... | layer_... | layer_... | layer_... | learni... |  neurons  | num_la... | patience  |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
| 1         | -2.475e+1 | 207.3     | 0.4754    | 392.8     | 166.1     | 66.95     | 66.94     | 45.01     | 226.0     | 0.6051    | 190.6     | 1.082     | 49.1      |
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
| 2         | -2.56e+13 | 421.6     | 0.1062    | 172.7     | 73.08     | 100.2     | 149.5     | 128.8     | 97.24     | 0.6157    | 63.25     | 2.169     | 30.99     |
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
| 3         | -2.439e+1 | 245.4     | 0.3926    | 179.9     | 147.2     | 164.7     | 42.4      | 168.1     | 70.2      | 0.0744    | 244.6     | 4.863     | 44.25     |
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms

In [426]:
batch_size = int(best_params['batch_size'])
dropout_rate = best_params['dropout_rate']
epochs = int(best_params['epochs'])
neurons = []
neurons.append(int(best_params['neurons']))
neurons.append(int(best_params['layer_neurons_1']))
neurons.append(int(best_params['layer_neurons_2']))
neurons.append(int(best_params['layer_neurons_3']))
neurons.append(int(best_params['layer_neurons_4']))
neurons.append(int(best_params['layer_neurons_5']))
learning_rate = best_params['learning_rate']
num_layers = int(best_params['num_layers'])
patience = int(best_params['patience'])

In [427]:
model = Sequential()

model.add(Dense(name = 'Dense1', units = neurons[0], input_dim = X_train.shape[1], activation = 'relu'))
model.add(Dropout(name = 'Dropout1', rate = dropout_rate))

for i in range(1, num_layers + 1):
    model.add(Dense(name = f'Dense{i + 1}', units = neurons[i], activation = 'relu'))
    model.add(Dropout(name = f'Dropout{i + 1}', rate = dropout_rate))

model.add(Dense(name = 'Output', units = 1, activation = 'linear'))

optimizer = Adam(learning_rate = learning_rate)

model.compile(optimizer = optimizer, loss = 'mean_squared_error')

model.summary()

Model: "sequential_147"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Dense1 (Dense)                  │ (None, 241)            │        11,086 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dropout1 (Dropout)              │ (None, 241)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dense2 (Dense)                  │ (None, 153)            │        37,026 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dropout2 (Dropout)              │ (None, 153)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dense3 (Dense)                  │ (None, 166)            │        25,564 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dropout3 (Dropout)              │ (None, 166)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Output (Dense)                  │ (None, 1)              │           167 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 73,843 (288.45 KB)

 Trainable params: 73,843 (288.45 KB)

 Non-trainable params: 0 (0.00 B)

In [428]:
es = EarlyStopping(monitor = 'val_loss', patience = int(patience), restore_best_weights = True)

history = model.fit(X_train, y_train, validation_split = .2, batch_size = batch_size, epochs = epochs, callbacks = es, verbose = 2)

Epoch 1/239
14/14 - 4s - 255ms/step - loss: 49057103872000.0000 - val_loss: 33228492636160.0000
Epoch 2/239
14/14 - 0s - 20ms/step - loss: 32890568048640.0000 - val_loss: 26520038932480.0000
Epoch 3/239
14/14 - 0s - 18ms/step - loss: 28319122718720.0000 - val_loss: 22687787253760.0000
Epoch 4/239
14/14 - 0s - 20ms/step - loss: 26862208155648.0000 - val_loss: 22439740309504.0000
Epoch 5/239
14/14 - 0s - 21ms/step - loss: 26007465623552.0000 - val_loss: 22462194515968.0000
Epoch 6/239
14/14 - 0s - 23ms/step - loss: 26183909507072.0000 - val_loss: 21497309560832.0000
Epoch 7/239
14/14 - 0s - 23ms/step - loss: 24856577966080.0000 - val_loss: 21422332182528.0000
Epoch 8/239
14/14 - 0s - 22ms/step - loss: 25188561321984.0000 - val_loss: 21551751626752.0000
Epoch 9/239
14/14 - 0s - 21ms/step - loss: 25008126558208.0000 - val_loss: 21361967759360.0000
Epoch 10/239
14/14 - 0s - 20ms/step - loss: 23877331714048.0000 - val_loss: 21563913011200.0000
Epoch 11/239
14/14 - 0s - 20ms/step - loss: 2360

In [429]:
test_preds = model.predict(X_test)
train_preds = model.predict(X_train)

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


In [430]:
train_rmse = mean_squared_error(y_train, train_preds)
test_rmse = mean_squared_error(y_test, test_preds)

print('Deep Neural Network Metrics:')
print(f'Train RMSE: {np.sqrt(train_rmse)}')
print(f'Test RMSE: {np.sqrt(test_rmse)}')

Train RMSE: 3645730.9615196977
Test RMSE: 5007409.22451356


In [401]:
# save the model, pca, and preprocessor so that new data can be fit using the same criteria
#model.save('dnn/dnn_pca45_model.keras')
#joblib.dump(pca, 'dnn/pca45.joblib')
#joblib.dump(preprocessor, 'dnn/preprocssor.joblib')

['dnn/preprocssor.joblib']

In [402]:
#loaded = tf.keras.models.load_model('dnn/dnn_pca45_model.keras')
#pca_loaded = joblib.load('dnn/pca45.joblib')
#load_prep = joblib.load('dnn/preprocssor.joblib')